In [6]:
import pickle
import pandas as pd

In [17]:
with open("db_songs_clustered.txt", "rb") as f:   # Unpickling
    db_songs = pickle.load(f)
    
with open("top_songs.txt", "rb") as f:   # Unpickling
    top_songs = pickle.load(f)

In [3]:
knn = pickle.load(open('model.p','rb'))

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

secrets_file = open("secrets.txt","r")
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]
        
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [59]:
from random import randint

def gnod (data=top_songs):
    song = input("Enter your song: ")
    artist = input("Enter your artist: ")
    # searching the song in our Hot songs list
    for i in range(len(top_songs)):
        # if it's in our list, then recommend another song from the list
        if ((top_songs['title'][i] == song) & (top_songs['artist'][i] == artist)):
            rand_number = randint(0,len(top_songs))
            return(print("I would recommend you to listen to: \"", top_songs['title'][rand_number], "\" by", top_songs['artist'][rand_number]))
    # if is not a Hot song 
    results = sp.search(q=song, limit=50)
    for i in range(len(results["tracks"]["items"])):
        if ((results["tracks"]["items"][i]["artists"][0]["name"]) == (artist)):
            uri = results["tracks"]["items"][i]["uri"]
            audio = pd.DataFrame(sp.audio_features('spotify:track:2JiHR4gCDjXwYVRDmwdk95'))[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
            
            y_pred_pca = knn.predict(X_test_pca)
            
            agafar model
            predir cluster per audio
            triar una canço del mateix cluster
            
            return(audio)
    return("Not in the list")

gnod()

Enter your song: Tides
Enter your artist: Ed Sheeran


[{'danceability': 0.631,
  'energy': 0.825,
  'key': 11,
  'loudness': -3.971,
  'mode': 1,
  'speechiness': 0.0433,
  'acousticness': 0.0167,
  'instrumentalness': 0,
  'liveness': 0.122,
  'valence': 0.427,
  'tempo': 129.942,
  'type': 'audio_features',
  'id': '2JiHR4gCDjXwYVRDmwdk95',
  'uri': 'spotify:track:2JiHR4gCDjXwYVRDmwdk95',
  'track_href': 'https://api.spotify.com/v1/tracks/2JiHR4gCDjXwYVRDmwdk95',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2JiHR4gCDjXwYVRDmwdk95',
  'duration_ms': 195760,
  'time_signature': 3}]

In [18]:
audio = pd.Dataframe(sp.audio_features('spotify:track:2JiHR4gCDjXwYVRDmwdk95'))
audio

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [19]:
import pandas as pd

audio = pd.DataFrame(sp.audio_features('spotify:track:2JiHR4gCDjXwYVRDmwdk95'))[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]

In [30]:

audio['key'] = audio['key'].astype(float)
audio['mode'] = audio['mode'].astype(float)
audio['instrumentalness'] = audio['instrumentalness'].astype(float)
audio['duration_ms'] = audio['duration_ms'].astype(float)
audio['time_signature'] = audio['time_signature'].astype(float)


danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms         float64
time_signature      float64
dtype: object

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y_scaled = scaler.fit_transform(audio)

y_pred_pca = knn.predict(scaler.fit_transform(audio))

ValueError: query data dimension must match training data dimension